In [ ]:
import logging
import json
import gc
from os import system
from glob import glob
from os.path import join
from pickle import dump, load
from datetime import datetime
from collections import namedtuple


import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import transformers
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
import optuna 

from arabert.preprocess import ArabertPreprocessor

import tensorboard_analysis 
from utilities import *

Error: Session cannot generate requests

# Parameters

In [ ]:
# Config
seed: int = 1

# Data 
data_proportion: float = 1.0 # propotion of data to be loaded in df
load_data: bool = False 
save_data: bool = False
test_validation_proportion: float = 0.013 # test and validation proportion from df

# Model 
model_name = "aubmindlab/bert-large-arabertv2"
from_pretrained_classifier: bool = False
pretrained_classifier_name: str = "2021-12-05-train-0.898193359375"
    
# Preprocessing 
sequence_length: int = 32
tokenize_in_batches: bool = False # Helps reduce memory footprint

# Paths
code_folder_path: str = ""

# Training 
validation_size: int = 4096
batch_size: int = 64
learning_rate: float = 1e-5
epochs: int = 4
warmup_ratio: float = 0.2
save_model_while_training: bool = True # maybe doesn't work, transformers is terrible
do_warmup: bool = True
eval_while_training: bool = True # maybe doesn't work, transformers is terrible
save_model_after_finish: bool = True # maybe doesn't work, transformers is terrible

# Etc
open_tensorboard: bool = True

## Don't touch 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Config
device = torch.device("cuda")
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
logging.disable(logging.WARNING)

# General
date = str(datetime.today().date())

# Data
labels = ["EGY", "GLF", "IRQ", "LEV", "NOR"]

# Preprocessing
arabert_prep = ArabertPreprocessor(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model 
if from_pretrained_classifier:
    pretrained_classifier_path = join(code_folder_path, "models", "finalized_models", pretrained_classifier_name)
    pretrained_classifier = AutoModelForSequenceClassification.from_pretrained(pretrained_classifier_path)

# Paths
train_path= f"./models/{date}-train"
search_path = f"./models/{date}-search"
dataset_string = "{}_dataset-seqlen" + str(sequence_length)


In [ ]:
assert 0 < data_proportion <= 1, "data_proportion must be right side inclusively between 0 and 1"
assert 0 <= warmup_ratio <= 1, "warmup_ratio must be inclusively between 0 and 1"
assert 0 < test_validation_proportion < 1, "test_validation_proportion must be exclusively between 0 and 1"
assert 0 < sequence_length, "sequence_length must be positive"
assert 0 < epochs, "epochs must be positive"
assert 0 < batch_size, "batch_size must be positive"
assert 0 < validation_size, "validation_size must be positive"


# Data (loading, preprocessing, tokenizing)

In [ ]:
%%time
# Date to dataframe ~(2.9 s)
df = get_SMADC_folder_data(code_folder_path)
df = df.sample(frac=data_proportion)

# Encode Y ~(307 ms)
classes = df["Region"].unique()
num_labels = len(classes)
class_to_index = {class_:index for class_, index in zip(classes, range(len(classes)))}
index_to_class = {index:class_ for class_, index in zip(classes, range(len(classes)))}
df["Labels"] = df["Region"].apply(class_to_index.get)

Wall time: 4.85 s


### Further preprocessing

In [ ]:
if not load_data:
    # Preprocess X ~(16min 22s)
    df["Text"] = df["Text"].apply(arabert_prep.preprocess)

    # split and ~(323ms)
    train, test = train_test_split(df, test_size=test_validation_proportion, random_state=seed)
    validate, test = train_test_split(test, test_size=len(test)-validation_size, random_state=seed)
    train.reset_index(drop=True, inplace=True)
    validate.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    # Tokenize
    if tokenize_in_batches:
        validate_encoding = batch_tokenize(tokenizer, validate["Text"], 10, sequence_length)
        test_encoding = batch_tokenize(tokenizer, test["Text"], 100, sequence_length)
        train_encoding = batch_tokenize(tokenizer, train["Text"], 500, sequence_length)
    else:
        validate_encoding = tokenize(tokenizer, validate["Text"].to_list(), sequence_length)
        test_encoding = tokenize(tokenizer, test["Text"].to_list(), sequence_length)
        train_encoding = tokenize(tokenizer, list(train["Text"]), sequence_length)

    # Make Dataset 
    validate_dataset = Dialect_dataset(validate_encoding, validate["Labels"].to_list())
    test_dataset = Dialect_dataset(test_encoding, test["Labels"].to_list())
    train_dataset = Dialect_dataset(train_encoding, train["Labels"].to_list())

# Save & load preprocessed data

In [ ]:
if save_data:
    save_preprocessed_data(validate_dataset, "preprocessed_validation")
    save_preprocessed_data(test_dataset, "preprocessed_test")
    save_preprocessed_data(train_dataset, "preprocessed_train")

if load_data:
    # ~(3mins)
    validate_dataset = load_preprocessed_data("preprocessed_validation")
    test_dataset = load_preprocessed_data("preprocessed_test")
    train_dataset = load_preprocessed_data("preprocessed_train")

# Training

In [ ]:
training_args = generate_training_args(
                    train_path, epochs=epochs, do_warmup=do_warmup, warmup_ratio=warmup_ratio, 
                    save_model=save_model_while_training, eval_while_training=eval_while_training, 
                    learning_rate=learning_rate,batch_size=batch_size, 
                    train_dataset_length=len(train_dataset), seed=seed
                    )

# Begin train

In [ ]:
if from_pretrained_classifier:
    trainer = Trainer(
    pretrained_classifier,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )
else:
    trainer = Trainer(
        model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validate_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )

In [ ]:
if open_tensorboard:
    from tensorboard import program
    tb = program.TensorBoard()
    tb.configure(argv=[None, '--logdir', join(code_folder_path, f"models")])
    print(f"Tensorflow listening on {tb.launch()}")

Tensorflow listening on http://localhost:6006/


In [ ]:
class EvaluatePleaseCallback(transformers.TrainerCallback):
    def on_log(self, args, state, control, model, **kwargs):
        print(trainer.evaluate())
 
gc.collect()
torch.cuda.empty_cache()
trainer.add_callback(EvaluatePleaseCallback())
trainer.train()

 12%|█▏        | 10000/86888 [1:13:27<9:21:08,  2.28it/s]

{'loss': 0.7872, 'learning_rate': 5.7484692233322605e-06, 'epoch': 0.46}


ERROR:tensorboard:File models\2022-03-28-train\runs\Mar28_02-36-09_At-Low-desktop\events.out.tfevents.1648424196.At-Low-desktop.8200.0 updated even though the current file is models\2022-03-28-train\runs\Mar28_02-36-09_At-Low-desktop\events.out.tfevents.1648424196.At-Low-desktop.8200.2

 12%|█▏        | 10000/86888 [1:13:35<9:21:08,  2.28it/s]

{'eval_loss': 0.5382239818572998, 'eval_macro_f1': 0.7711108669405669, 'eval_macro_precision': 0.7871137170759666, 'eval_macro_recall': 0.758862307953288, 'eval_accuracy': 0.808349609375, 'eval_report': {'NOR': {'precision': 0.6753246753246753, 'recall': 0.7086882453151618, 'f1-score': 0.6916043225270159, 'support': 587}, 'IRQ': {'precision': 0.8582554517133957, 'recall': 0.924496644295302, 'f1-score': 0.8901453957996768, 'support': 1788}, 'LEV': {'precision': 0.7905918057663126, 'recall': 0.7186206896551725, 'f1-score': 0.7528901734104047, 'support': 725}, 'EGY': {'precision': 0.806615776081425, 'recall': 0.6788008565310493, 'f1-score': 0.7372093023255814, 'support': 467}, 'GLF': {'precision': 0.8047808764940239, 'recall': 0.7637051039697542, 'f1-score': 0.7837051406401552, 'support': 529}, 'accuracy': 0.808349609375, 'macro avg': {'precision': 0.7871137170759666, 'recall': 0.758862307953288, 'f1-score': 0.7711108669405669, 'support': 4096}, 'weighted avg': {'precision': 0.80726905330


 12%|█▏        | 10000/86888 [1:13:42<9:21:08,  2.28it/s]

...

{'eval_loss': 0.5382239818572998, 'eval_macro_f1': 0.7711108669405669, 'eval_macro_precision': 0.7871137170759666, 'eval_macro_recall': 0.758862307953288, 'eval_accuracy': 0.808349609375, 'eval_report': {'NOR': {'precision': 0.6753246753246753, 'recall': 0.7086882453151618, 'f1-score': 0.6916043225270159, 'support': 587}, 'IRQ': {'precision': 0.8582554517133957, 'recall': 0.924496644295302, 'f1-score': 0.8901453957996768, 'support': 1788}, 'LEV': {'precision': 0.7905918057663126, 'recall': 0.7186206896551725, 'f1-score': 0.7528901734104047, 'support': 725}, 'EGY': {'precision': 0.806615776081425, 'recall': 0.6788008565310493, 'f1-score': 0.7372093023255814, 'support': 467}, 'GLF': {'precision': 0.8047808764940239, 'recall': 0.7637051039697542, 'f1-score': 0.7837051406401552, 'support': 529}, 'accuracy': 0.808349609375, 'macro avg': {'precision': 0.7871137170759666, 'recall': 0.758862307953288, 'f1-score': 0.7711108669405669, 'support': 4096}, 'weighted avg': {'precision': 0.80726905330


 12%|█▏        | 10000/86888 [2:04:05<9:21:08,  2.28it/s]

{'eval_loss': 0.5382239818572998, 'eval_macro_f1': 0.7711108669405669, 'eval_macro_precision': 0.7871137170759666, 'eval_macro_recall': 0.758862307953288, 'eval_accuracy': 0.808349609375, 'eval_report': {'NOR': {'precision': 0.6753246753246753, 'recall': 0.7086882453151618, 'f1-score': 0.6916043225270159, 'support': 587}, 'IRQ': {'precision': 0.8582554517133957, 'recall': 0.924496644295302, 'f1-score': 0.8901453957996768, 'support': 1788}, 'LEV': {'precision': 0.7905918057663126, 'recall': 0.7186206896551725, 'f1-score': 0.7528901734104047, 'support': 725}, 'EGY': {'precision': 0.806615776081425, 'recall': 0.6788008565310493, 'f1-score': 0.7372093023255814, 'support': 467}, 'GLF': {'precision': 0.8047808764940239, 'recall': 0.7637051039697542, 'f1-score': 0.7837051406401552, 'support': 529}, 'accuracy': 0.808349609375, 'macro avg': {'precision': 0.7871137170759666, 'recall': 0.758862307953288, 'f1-score': 0.7711108669405669, 'support': 4096}, 'weighted avg': {'precision': 0.80726905330


 12%|█▏        | 10000/86888 [2:04:12<9:21:08,  2.28it/s]

{'eval_loss': 0.5382239818572998, 'eval_macro_f1': 0.7711108669405669, 'eval_macro_precision': 0.7871137170759666, 'eval_macro_recall': 0.758862307953288, 'eval_accuracy': 0.808349609375, 'eval_report': {'NOR': {'precision': 0.6753246753246753, 'recall': 0.7086882453151618, 'f1-score': 0.6916043225270159, 'support': 587}, 'IRQ': {'precision': 0.8582554517133957, 'recall': 0.924496644295302, 'f1-score': 0.8901453957996768, 'support': 1788}, 'LEV': {'precision': 0.7905918057663126, 'recall': 0.7186206896551725, 'f1-score': 0.7528901734104047, 'support': 725}, 'EGY': {'precision': 0.806615776081425, 'recall': 0.6788008565310493, 'f1-score': 0.7372093023255814, 'support': 467}, 'GLF': {'precision': 0.8047808764940239, 'recall': 0.7637051039697542, 'f1-score': 0.7837051406401552, 'support': 529}, 'accuracy': 0.808349609375, 'macro avg': {'precision': 0.7871137170759666, 'recall': 0.758862307953288, 'f1-score': 0.7711108669405669, 'support': 4096}, 'weighted avg': {'precision': 0.80726905330

# Analysis

In [ ]:
evaluation = trainer.evaluate(test_dataset)
evaluation

Error: Session cannot generate requests

# Save & load

In [ ]:
if save_model_after_finish:
    save_path = f'models/finalized_models/{trainer.args.output_dir.split("/")[-1]}-{model_name[model_name.rfind("/")+1:]}-{trainer.evaluate()["eval_accuracy"]}'
    trainer.save_model(save_path)
    with open(join(save_path, "evaluation.json"), "w") as file:
        file.write(json.dumps(evaluation))
        
    with open(join(save_path, "arguments.txt"), "w") as file:
        file.write(str(training_args))
    

100%|██████████| 64/64 [00:07<00:00,  8.74it/s]


# Hyperparameter search

In [ ]:
# def objective(trial: optuna.Trial):     
    
#     training_args = generate_training_args(search_path, epochs=None, do_warmup=False, save_model=False, eval_while_training=False)
#     training_args.learning_rate= trial.suggest_loguniform('learning_rate', low=4e-5, high=0.01)
#     training_args.weight_decay= trial.suggest_loguniform('weight_decay', 4e-5, 0.01)
#     training_args.num_train_epochs= trial.suggest_int('num_train_epochs', low=2, high=5)
    
#     trainer = Trainer(
#         model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=validate_dataset,
#         compute_metrics=compute_metrics,
#         callbacks=[TensorBoardCallback()]
#     )
    
#     result = trainer.train()     
    
#     return result.training_loss # Or result.training_loss["metric_name"] ps: change direction in study if necessary
   
# We want to minimize the loss! 
# study = optuna.create_study(study_name='hyper-parameter-search', direction='minimize') 
# study.optimize(func=objective, n_trials=15)

# print(study.best_value) 
# print(study.best_params) 
# print(study.best_trial)

# trainer.hyperparameter_search(n_trials=10)
